## Goal
* Gather sunset and sunrise times from an array of dates and locations and store results

https://sunrise-sunset.org/api

In [229]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd
import time
import os.path

## Build Date Array

In [5]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range
start_date = date(2017, 7, 1)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [241]:
print(dates[:4])
print(dates[-4:])

['2017-07-01', '2017-07-02', '2017-07-03', '2017-07-04']
['2017-10-28', '2017-10-29', '2017-10-30', '2017-10-31']


## Load Location Dictionary

In [8]:
# Load station location csv into a DataFrame
stations = pd.read_csv('../output/weather_monitoring-locations.csv')

In [242]:
stations.head()

,id,lat,lon,neighborhood,city
0,KMTCORVA9,46.277557,-114.045059,Hamilton Heights,Corvallis
1,KMTCORVA10,46.284927,-114.106499,All View Lane,Corvallis
2,KMTCORVA12,46.323433,-114.058075,Soft Rock,Corvallis
3,KMTCORVA15,46.314114,-114.119720,Corvallis - West Edge of Town,Corvallis
4,KMTFLORE4,46.716743,-114.007782,MPG Ranch - Baldy Draw,Florence


## Build URLs

In [31]:
# Create empty data frame
df = pd.DataFrame(columns=['id','url'])

In [92]:
def build_url_df(stations, dates): 
    url_station = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            row_id = row['id']
            lat = row['lat']
            lon = row['lon']
            row_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date
            url_station.append(dict({'station_id': row_id, 'url': row_url, 'date': date}))
    return(url_station)

In [93]:
# Generate url list for api calls
url_batch = build_url_df(stations, dates)

In [246]:
print('Number of urls: ' + str(len(url_batch)))
print(url_batch[0])

Number of urls: 4182
{'station_id': 'KMTCORVA9', 'url': 'https://api.sunrise-sunset.org/json?lat=46.277557&lng=-114.045059&date=2017-07-01', 'date': '2017-07-01'}


## Call API and store results

In [217]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    
    return parsed_json['results']

In [234]:
def save_results(df_day):
    filepath = '../output/sunrise_sunset.csv'
    
    if os.path.isfile(filepath):
        with open(filepath, 'a') as f:
            df_day.to_csv(f, header=False, index=False)
    else:
        df_day.to_csv(filepath, index=False)

In [238]:
%%time
successfull_calls = 0

# Loop over url_batch to call api and store results
for url in url_batch[10:11]:
    # sleep to limit API call speed
    time.sleep(6)
    
    print(url['station_id'] + '\n' + url['url'])
    result = api_call(url['url'])
    
    # include station_id
    result['station_id'] = url['station_id']
    
    # include date 
    result['date'] = url['date']
    
    df_day = pd.DataFrame.from_dict([result])
    save_results(df_day)
    
    successfull_calls += 1
    print('successfull_calls: ' + str(successfull_calls))

KMTCORVA9
https://api.sunrise-sunset.org/json?lat=46.277557&lng=-114.045059&date=2017-07-11
successfull_calls: 1
CPU times: user 20.3 ms, sys: 3.89 ms, total: 24.2 ms
Wall time: 6.4 s
